In [ ]:
import glob
import pandas as pd
from econtools.metrics import reg
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client, progress
import multiprocess as mp
import matplotlib.pyplot as plt
import json
import dask
import geopandas as gp
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
from multiprocessing.pool import ThreadPool, Pool
from dask import delayed
from dask import compute

def haversine(row):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lon1, lat1, lon2, lat2 = row
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

 ### 1.Read in every year of available data (2012 to 2020) using Dask

In [ ]:
cd "C:\Users\27354\Desktop\ace 592 file"

In [ ]:
import pandas as pd

In [ ]:
from dask.distributed import Client, progress
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')
client

In [ ]:
t: year
i: exporter
j: importer
k: product
v: value 
q: quantity

In [ ]:
import dask
import dask.dataframe as dd

In [ ]:
df = dd.read_csv("BACI_HS12*.csv")
df

In [ ]:
df.head(5)

In [ ]:
df.t.i.j.k.q.mean().compute()
df.npartitions

In [ ]:
df.partitions[5].compute()
type(df.partitions[5].compute())
df.known_divisions
df = df.set_index('t', sorted=True)
df = df.set_index('i', sorted=True)
df = df.set_index('j', sorted=True)
df = df.set_index('k', sorted=True)
df = df.set_index('v', sorted=True)
df = df.set_index('q', sorted=True)

In [ ]:
df.known_divisions
df.head()
df.info()
df.divisions

##In the year 2012, who were the top 10 countries with the most trading partners? What about the bottom 10?

In [ ]:
df.loc[2012].compute()
len(df)

In [ ]:
df12 = pd.read_csv("P:trade_data/BACI_HS12_Y2018_V202001.csv",nrows=100)
df12

In [ ]:
import dask.dataframe as dd
df = dd.read_csv("C:\Users\27354\Desktop\ace 592 file\ace 592 file\BACI_HS12_V202201\BACI_HS12_Y*.csv")
df

In [ ]:
import glob
glob.glob("C:\Users\27354\Desktop\ace 592 file\ace 592 file\BACI_HS12_V202201\BACI_HS12_Y*.csv")

In [ ]:
df.loc[df.t==2012]

In [ ]:
df.loc[df.t==2012].groupby("q")['v'].mean().compute()

In [ ]:
df_2012 = df.loc[2012]

In [ ]:
top_12 = df_2012.v.value_counts().nlargest(10).compute()

In [ ]:
top_12

In [ ]:
top_12df=top_12.to_frame()

In [ ]:
top_12df_merged=top_12df.merge(value, how='left', right_index=True, left_index=True)\
.sort_values(by='v''q', ascending=False)
top_12df_merged

In [ ]:
def most_exports(partition):
    # partition is a pandas.DataFrame
    country = partition.country.value_counts().nlargest(1).index[0]
    description = country_mapping.loc[minfd].minfddesc.iloc[0]
    year = int(partition.index[0])
    return pd.DataFrame({'year': [year], 'description': [description]})

In [ ]:
country_year = df.map_partitions(most_exports, 
                              meta={'year': int,
                                    'description': str})

In [ ]:
mnfd_year.compute()

In [ ]:
products = pd.read_csv("C:\Users\27354\Desktop\ace 592 file\ace 592 fileproduct_codes_HS12_V202001.csv")
products = products.rename(columns={"code":"k"})

In [ ]:
products

In [ ]:
df_wdes = df.merge(products)

In [ ]:
countries = pd.read_csv("C:\Users\27354\Desktop\ace 592 file\ace 592 file/country_codes_V202001.csv",encoding = "ISO-8859-1")
countries = countries.rename(columns={"country_code":"i","country_name_full":"name_i"})
df_wdes = df_wdes.merge(countries[['i','name_i']])

countries = pd.read_csv("C:\Users\27354\Desktop\ace 592 file\ace 592 file/country_codes_V202001.csv",encoding = "ISO-8859-1")
countries = countries.rename(columns={"country_code":"j","country_name_full":"name_j"})
df_wdes = df_wdes.merge(countries[['j','name_j']])

In [ ]:
df_wdes.head()

In [ ]:
df_wdes['avg_price'] = df_wdes['v']/df_wdes['q']

In [ ]:
df_wdes.head(10)

In [ ]:
avg12 = df.loc[df.t==2012]['avg_price']

In [ ]:
ex = avg12.compute()

In [ ]:
ex

In [ ]:
avg12.mean().compute()

In [ ]:
avg12.var().compute()

In [ ]:
product_counts = df['k'].value_counts().compute()

product_dict = products.set_index("k")['description'].to_dict()

product_counts.index=  pd.Series(product_counts.index).replace(product_dict)

In [ ]:
product_counts.head(10)

In [ ]:
df['q'].nlargest(10).compute()

In [ ]:
country_v_sums = df.groupby("i")['v'].sum().compute()

In [ ]:
country_v_sums.sort_values(ascending=False)

### An analyst asks you "for which commodity sectors is distance most important in terms of exporting?" Describe what your economic intuition would say about the answer, and then also map out an analysis strategy for answering their question.

First of all, to solve the problem, the data I need is the data about the countries exporting bulk commodities, the specific classification and proportion of export products. At the same time, it also needs the distance requirements of the exporting countries and the economic organization structure of the exporting countries, so as to judge the country's tolerance and demand for imported products.

For the use of data, I will follow two aspects: the use of direct evidence and the auxiliary verification of indirect evidence. Therefore, I will first gather direct evidence, including the content and quantity of exports and the proportion of exports in the national GDP, while indirect evidence means the total economic volume, economic structure and import proportion of the exported country, so as to judge whether it can be used as a high-quality cooperative trade object.